# Affaan's Bitcoin Price Predictor Using Time-Series and SVR / Random Forest

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

In [ ]:
# Read 5 Year Price/Time Daily Increment Data for Bitcoin into a DataFrame
bitcoin = bitcoin = pd.read_csv('BTC5Y.csv')
bitcoin.head()

In [ ]:
bitcoin.info()

In [ ]:
# Convert the "Date" column to a datetime object
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])

# Set the float format to show up to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

# Plot a scatter plot of Bitcoin price data with time in years on the x-axis and price on the y-axis
plt.plot(bitcoin['Date'], bitcoin['Close'])

# Add a title and labels to the plot
plt.title('Bitcoin Price Scatter Plot')
plt.xlabel('Year')
plt.ylabel('Price')

# Show the plot
plt.show()

In [ ]:
# Set the date as the index
bitcoin = bitcoin.set_index('Date')

# Split the data into training and testing sets
train_size = int(len(bitcoin) * 0.8)
train_data = bitcoin[:train_size]
test_data = bitcoin[train_size:]

# Scale the data
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Split the data into features (X) and target (y)
X_train = train_data[:, :-1]
y_train = train_data[:, -1]
X_test = test_data[:, :-1]
y_test = test_data[:, -1]

# Define the Neural Network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model on the training set
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Scale the predictions back to the original scale
y_pred = scaler.inverse_transform(np.concatenate((X_test, y_pred), axis=1))[:, -1]
y_test = scaler.inverse_transform(np.concatenate((X_test, y_test.reshape(-1, 1)), axis=1))[:, -1]

# Evaluate the model performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', rmse)

# Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Plot the actual data and the predictions
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(bitcoin.index, bitcoin['Close'], label='Actual')
ax.plot(test_data[:, -2], y_pred, label='Predicted')
ax.legend(loc='upper left')
plt.title('Bitcoin Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()
